In [ ]:
import pandas as pd

In [ ]:
qrels_file = "../data/external/qrels2022.txt"

In [ ]:
df = pd.read_csv(qrels_file, sep=' ', names=['topic', 'skip','trial', 'qrel'])

In [ ]:
for qrel in [0, 1, 2]:
    print(qrel, len(df[df['qrel'] == qrel]))

In [ ]:
print(f"unique trials: {len(df['trial'].unique())}")

In [ ]:
# trials which appear in the most number of patients
df['trial'].value_counts()

In [ ]:
threshold = len(df['topic'].unique()) - 5
# trials which appear in the most number of patients AND are relevant or excluded
df[(df['trial'].isin(df['trial'].value_counts()[df['trial'].value_counts() > threshold].index)) & df['qrel'].isin([1,2])]['trial'].value_counts()

# stats for patients

In [ ]:
from CTnlp.patient import load_patients_from_xml
import numpy as np

In [ ]:
p21 = load_patients_from_xml("../data/external/topics2021.xml")
p22 = load_patients_from_xml("../data/external/topics2022.xml")

In [ ]:
np.mean([len(x.description.split('.')) for x in p21])

In [ ]:
np.mean([len(x.description.split('.')) for x in p22])

In [ ]:
np.mean([len(x.description.split(' ')) for x in p21])

In [ ]:
np.mean([len(x.description.split(' ')) for x in p22])